In [80]:
%load_ext pypath_magic
%pypath -a /home/vlad/projects/datamanagers/daily/

The pypath_magic extension is already loaded. To reload it, use:
  %reload_ext pypath_magic


UsageError: '/home/vlad/projects/datamanagers/daily' is already in the user path.


In [81]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [145]:
import numpy as np
import scipy as sp
import pandas as pd
import xarray as xr
import statsmodels.api as sm
from pathlib import Path
import time
import sqlite3

# sklearn imports
from sklearn.linear_model import LinearRegression

from daily.datamanager import DailyDataManager
import plotter.activate
from plotter import plot
import matplotlib.pyplot as plt
import seaborn as sns

In [146]:
#  execution time tool
COMMAND_START_TIME = None

def tic():
    global COMMAND_START_TIME
    COMMAND_START_TIME = time.time()

def toc(name='Execution time'):
    global COMMAND_START_TIME
    print(f'\n ---------- {name}:  {time.time() - COMMAND_START_TIME} seconds \n')
    COMMAND_START_TIME = None

In [147]:
from numpy.random import default_rng
rng = default_rng()

In [148]:
nb_rows = 1000000
data = pd.DataFrame(rng.standard_normal(size=(nb_rows, 10)))
print('dataframe size: ', data.memory_usage(0).sum()/1e6, 'MB')
data.index = pd.date_range('2010-01-01', periods=nb_rows, freq='5min')
data.columns = list('ABCDEFJHKL')
data

dataframe size:  80.0 MB


,A,B,C,D,E,F,J,H,K,L
2010-01-01 00:00:00,-0.760247,-1.260593,0.436157,2.317471,1.264199,-0.852664,-0.629201,0.691575,0.112751,0.152739
2010-01-01 00:05:00,-0.923852,1.062645,-1.401953,0.201615,0.994345,-0.577708,-0.279195,-0.541942,0.727288,-0.862150
2010-01-01 00:10:00,1.040367,-0.921304,-1.060270,0.612874,-0.141591,0.781121,2.230413,1.173031,-0.458583,-0.861787
2010-01-01 00:15:00,0.506024,1.057408,-0.167510,0.388773,-1.699381,0.769543,0.700072,-0.456726,-0.110524,-1.376078
2010-01-01 00:20:00,-0.653676,-1.089360,-0.857728,-1.216899,-0.198680,1.381250,0.234413,-1.564884,-1.377331,-0.826748
...,...,...,...,...,...,...,...,...,...,...
2019-07-05 04:55:00,-0.323583,0.218284,0.276085,-0.432085,-0.261821,-0.291897,1.793808,-0.414312,0.353982,-0.055002
2019-07-05 05:00:00,-1.749736,0.811144,0.652605,-0.580753,-0.096323,1.827338,0.777880,1.711257,1.414369,0.342793
2019-07-05 05:05:00,0.595135,-0.735942,-0.645187,0.284433,-2.210899,1.613154,-0.047682,-0.687121,0.501155,-0.632339
2019-07-05 05:10:00,0.706281,-0.698677,0.140545,0.071052,0.478083,-0.004968,1.274431,-2.128788,0.232280,0.835773


In [149]:
data_root = Path('/home/vlad/projects/pythonguides/notebooks/data')

In [150]:
data['A'] = data['A'].astype(np.float32)
print(data.dtypes)
data.index.name = 'date'
data.columns.name = 'field'
data

A    float32
B    float64
C    float64
D    float64
E    float64
F    float64
J    float64
H    float64
K    float64
L    float64
dtype: object


field,A,B,C,D,E,F,J,H,K,L
date,,,,,,,,,,
2010-01-01 00:00:00,-0.760247,-1.260593,0.436157,2.317471,1.264199,-0.852664,-0.629201,0.691575,0.112751,0.152739
2010-01-01 00:05:00,-0.923852,1.062645,-1.401953,0.201615,0.994345,-0.577708,-0.279195,-0.541942,0.727288,-0.862150
2010-01-01 00:10:00,1.040367,-0.921304,-1.060270,0.612874,-0.141591,0.781121,2.230413,1.173031,-0.458583,-0.861787
2010-01-01 00:15:00,0.506024,1.057408,-0.167510,0.388773,-1.699381,0.769543,0.700072,-0.456726,-0.110524,-1.376078
2010-01-01 00:20:00,-0.653676,-1.089360,-0.857728,-1.216899,-0.198680,1.381250,0.234413,-1.564884,-1.377331,-0.826748
...,...,...,...,...,...,...,...,...,...,...
2019-07-05 04:55:00,-0.323583,0.218284,0.276085,-0.432085,-0.261821,-0.291897,1.793808,-0.414312,0.353982,-0.055002
2019-07-05 05:00:00,-1.749736,0.811144,0.652605,-0.580753,-0.096323,1.827338,0.777880,1.711257,1.414369,0.342793
2019-07-05 05:05:00,0.595135,-0.735942,-0.645187,0.284433,-2.210899,1.613154,-0.047682,-0.687121,0.501155,-0.632339


### csv

In [151]:
tic()
data.to_csv(data_root/'data.csv')
toc('Save to csv')


 ---------- Save to csv:  11.172499656677246 seconds 



In [152]:
tic()
data_read = pd.read_csv(data_root/'data.csv', usecols=list('ACE'), nrows=20)
toc('Read csv, specify columns and number of rows')


 ---------- Read csv, specify columns and number of rows:  0.542391300201416 seconds 



In [153]:
data_read.dtypes  # cannot save metadata

A    float64
C    float64
E    float64
dtype: object

In [154]:
data_read

,A,C,E
0,-0.760247,0.436157,1.264199
1,-0.923852,-1.401953,0.994345
2,1.040367,-1.060270,-0.141591
3,0.506024,-0.167510,-1.699381
4,-0.653676,-0.857728,-0.198680
5,-0.060643,-1.815068,1.195094
6,0.015082,-1.158121,-0.493684
7,-1.173377,0.529912,-0.976268
8,-0.550549,0.341183,0.843439
9,1.166976,-0.719979,-1.882294


In [155]:
tic()
pd.read_csv(data_root/'data.csv')
toc('Read csv full')

,date,A,B,C,D,E,F,J,H,K,L
0,2010-01-01 00:00:00,-0.760247,-1.260593,0.436157,2.317471,1.264199,-0.852664,-0.629201,0.691575,0.112751,0.152739
1,2010-01-01 00:05:00,-0.923852,1.062645,-1.401953,0.201615,0.994345,-0.577708,-0.279195,-0.541942,0.727288,-0.862150
2,2010-01-01 00:10:00,1.040367,-0.921304,-1.060270,0.612874,-0.141591,0.781121,2.230413,1.173031,-0.458583,-0.861787
3,2010-01-01 00:15:00,0.506024,1.057408,-0.167510,0.388773,-1.699381,0.769543,0.700072,-0.456726,-0.110524,-1.376078
4,2010-01-01 00:20:00,-0.653676,-1.089360,-0.857728,-1.216899,-0.198680,1.381250,0.234413,-1.564884,-1.377331,-0.826748
...,...,...,...,...,...,...,...,...,...,...,...
999995,2019-07-05 04:55:00,-0.323583,0.218284,0.276085,-0.432085,-0.261821,-0.291897,1.793808,-0.414312,0.353982,-0.055002
999996,2019-07-05 05:00:00,-1.749736,0.811144,0.652605,-0.580753,-0.096323,1.827338,0.777880,1.711257,1.414369,0.342793
999997,2019-07-05 05:05:00,0.595135,-0.735942,-0.645187,0.284433,-2.210899,1.613154,-0.047682,-0.687121,0.501155,-0.632339
999998,2019-07-05 05:10:00,0.706281,-0.698677,0.140545,0.071052,0.478083,-0.004968,1.274431,-2.128788,0.232280,0.835773



 ---------- Read csv full:  1.5043094158172607 seconds 



### pickle

In [156]:
tic()
data.to_pickle(data_root/'data.pkl')
toc('Pickle time')


 ---------- Pickle time:  0.12399029731750488 seconds 



In [157]:
tic()
data_read = pd.read_pickle(data_root/'data.pkl')
toc('Pickle time')
data_read.dtypes


 ---------- Pickle time:  0.020023822784423828 seconds 



field
A    float32
B    float64
C    float64
D    float64
E    float64
F    float64
J    float64
H    float64
K    float64
L    float64
dtype: object

### parquet

In [158]:
tic()
data.to_parquet(data_root/'data.parquet', engine='pyarrow')
toc('Parquet')


 ---------- Parquet:  0.47109389305114746 seconds 



In [159]:
tic()
df = pd.read_parquet(data_root/'data.parquet')
toc('read parquet, specify columns')
print(df.dtypes)
df


 ---------- read parquet, specify columns:  0.13911914825439453 seconds 

field
A    float32
B    float64
C    float64
D    float64
E    float64
F    float64
J    float64
H    float64
K    float64
L    float64
dtype: object


field,A,B,C,D,E,F,J,H,K,L
date,,,,,,,,,,
2010-01-01 00:00:00,-0.760247,-1.260593,0.436157,2.317471,1.264199,-0.852664,-0.629201,0.691575,0.112751,0.152739
2010-01-01 00:05:00,-0.923852,1.062645,-1.401953,0.201615,0.994345,-0.577708,-0.279195,-0.541942,0.727288,-0.862150
2010-01-01 00:10:00,1.040367,-0.921304,-1.060270,0.612874,-0.141591,0.781121,2.230413,1.173031,-0.458583,-0.861787
2010-01-01 00:15:00,0.506024,1.057408,-0.167510,0.388773,-1.699381,0.769543,0.700072,-0.456726,-0.110524,-1.376078
2010-01-01 00:20:00,-0.653676,-1.089360,-0.857728,-1.216899,-0.198680,1.381250,0.234413,-1.564884,-1.377331,-0.826748
...,...,...,...,...,...,...,...,...,...,...
2019-07-05 04:55:00,-0.323583,0.218284,0.276085,-0.432085,-0.261821,-0.291897,1.793808,-0.414312,0.353982,-0.055002
2019-07-05 05:00:00,-1.749736,0.811144,0.652605,-0.580753,-0.096323,1.827338,0.777880,1.711257,1.414369,0.342793
2019-07-05 05:05:00,0.595135,-0.735942,-0.645187,0.284433,-2.210899,1.613154,-0.047682,-0.687121,0.501155,-0.632339


In [160]:
tic()
pd.read_parquet(data_root/'data.parquet')
toc('read parquet full')

field,A,B,C,D,E,F,J,H,K,L
date,,,,,,,,,,
2010-01-01 00:00:00,-0.760247,-1.260593,0.436157,2.317471,1.264199,-0.852664,-0.629201,0.691575,0.112751,0.152739
2010-01-01 00:05:00,-0.923852,1.062645,-1.401953,0.201615,0.994345,-0.577708,-0.279195,-0.541942,0.727288,-0.862150
2010-01-01 00:10:00,1.040367,-0.921304,-1.060270,0.612874,-0.141591,0.781121,2.230413,1.173031,-0.458583,-0.861787
2010-01-01 00:15:00,0.506024,1.057408,-0.167510,0.388773,-1.699381,0.769543,0.700072,-0.456726,-0.110524,-1.376078
2010-01-01 00:20:00,-0.653676,-1.089360,-0.857728,-1.216899,-0.198680,1.381250,0.234413,-1.564884,-1.377331,-0.826748
...,...,...,...,...,...,...,...,...,...,...
2019-07-05 04:55:00,-0.323583,0.218284,0.276085,-0.432085,-0.261821,-0.291897,1.793808,-0.414312,0.353982,-0.055002
2019-07-05 05:00:00,-1.749736,0.811144,0.652605,-0.580753,-0.096323,1.827338,0.777880,1.711257,1.414369,0.342793
2019-07-05 05:05:00,0.595135,-0.735942,-0.645187,0.284433,-2.210899,1.613154,-0.047682,-0.687121,0.501155,-0.632339



 ---------- read parquet full:  0.16373395919799805 seconds 



### netcdf

In [161]:
import xarray as xr
xrvar = xr.DataArray(data)

In [162]:
tic()
xrvar.to_netcdf(data_root/'data.ncdf')
toc('xarray NetCDF')

PermissionError: [Errno 13] Permission denied: b'/home/vlad/projects/pythonguides/notebooks/data/data.ncdf'

In [ ]:
tic()
xr.open_dataarray(data_root/'data.ncdf')
toc('Reading NetCDF')

### hdf5

In [ ]:
tic()
data.to_hdf(data_root/'data.h5', key='data', format='table', data_columns=True)
toc('HDF5 time')

In [ ]:
tic()
pd.read_hdf(data_root/'data.h5', 'data')
toc('read h5 full')

In [ ]:
tic()
pd.read_hdf(data_root/'data.h5', 'data', where='A > 0')
toc('read h5 full')

## more on HDF5

In [ ]:
s = pd.HDFStore(data_root/'store.h5')

In [ ]:
tic()
s['data'] = data
toc()

sqr_data = data**2
tic()
s['sqr'] = sqr_data
toc()

In [ ]:
import numpy as np
data = data.astype(np.float32)

In [163]:
data

field,A,B,C,D,E,F,J,H,K,L
date,,,,,,,,,,
2010-01-01 00:00:00,-0.760247,-1.260593,0.436157,2.317471,1.264199,-0.852664,-0.629201,0.691575,0.112751,0.152739
2010-01-01 00:05:00,-0.923852,1.062645,-1.401953,0.201615,0.994345,-0.577708,-0.279195,-0.541942,0.727288,-0.862150
2010-01-01 00:10:00,1.040367,-0.921304,-1.060270,0.612874,-0.141591,0.781121,2.230413,1.173031,-0.458583,-0.861787
2010-01-01 00:15:00,0.506024,1.057408,-0.167510,0.388773,-1.699381,0.769543,0.700072,-0.456726,-0.110524,-1.376078
2010-01-01 00:20:00,-0.653676,-1.089360,-0.857728,-1.216899,-0.198680,1.381250,0.234413,-1.564884,-1.377331,-0.826748
...,...,...,...,...,...,...,...,...,...,...
2019-07-05 04:55:00,-0.323583,0.218284,0.276085,-0.432085,-0.261821,-0.291897,1.793808,-0.414312,0.353982,-0.055002
2019-07-05 05:00:00,-1.749736,0.811144,0.652605,-0.580753,-0.096323,1.827338,0.777880,1.711257,1.414369,0.342793
2019-07-05 05:05:00,0.595135,-0.735942,-0.645187,0.284433,-2.210899,1.613154,-0.047682,-0.687121,0.501155,-0.632339


In [164]:
data.to_hdf(data_root/'data.h5', 'df')
df_read = pd.read_hdf(data_root/'data.h5', 'df')

In [165]:
s = pd.HDFStore(data_root/'data.h5')
s

<class 'pandas.io.pytables.HDFStore'>
File path: /home/vlad/projects/pythonguides/notebooks/data/data.h5

### SQLite3

In [174]:
tic()
try:
    con = sqlite3.connect(data_root/'data.db')
    data.to_sql(name='data', con=con, if_exists='replace')
finally:
    con.close()
toc('write data to SQLite database')


 ---------- write data to SQLite database:  4.598782300949097 seconds 



In [176]:
tic()
try:
    con = sqlite3.connect(data_root/'data.db')
    query = 'SELECT * FROM DATA WHERE A>0'
    df = pd.read_sql_query(query, con, index_col='date')
finally:
    con.close()
toc('read sqlit DB')
df


 ---------- read sqlit DB:  2.4474782943725586 seconds 



,A,B,C,D,E,F,J,H,K,L
date,,,,,,,,,,
2010-01-01 00:10:00,1.040367,-0.921304,-1.060270,0.612874,-0.141591,0.781121,2.230413,1.173031,-0.458583,-0.861787
2010-01-01 00:15:00,0.506024,1.057408,-0.167510,0.388773,-1.699381,0.769543,0.700072,-0.456726,-0.110524,-1.376078
2010-01-01 00:30:00,0.015082,1.178137,-1.158121,-0.262047,-0.493684,2.354985,1.091843,0.774348,0.420948,-0.818202
2010-01-01 00:45:00,1.166976,0.748087,-0.719979,-0.723594,-1.882294,-0.665924,0.137243,0.904219,-1.537825,-1.546916
2010-01-01 00:55:00,0.834031,-1.533501,0.301234,-0.537711,-1.157321,-2.219657,-0.613741,-0.436406,0.691445,-0.040178
...,...,...,...,...,...,...,...,...,...,...
2019-07-05 04:20:00,1.438379,1.645018,-1.394244,-0.853660,-0.058895,0.754862,-2.725054,0.856525,-0.858188,2.232440
2019-07-05 04:30:00,1.191332,-0.526539,-0.063953,1.264764,2.723535,1.913552,-0.743727,1.416377,0.745567,-2.129451
2019-07-05 04:45:00,0.913577,-0.526809,0.513852,-1.062577,-2.690554,0.872304,0.153217,-0.937961,0.512733,-0.177080


### comparison of file sizes

In [173]:
formats = ['csv', 'parquet', 'h5', 'ncdf', 'db']
file_sizes = {}
for f in formats:
    file_sizes[f] = (data_root/f'data.{f}').stat().st_size//1000000
print('File size in MB:')
pd.Series(file_sizes)
print(f'Original df size is ', data.memory_usage().sum()//1000000)

File size in MB:


csv        207
parquet     85
h5         241
ncdf        88
db         305
dtype: int64

Original df size is  84


### Reading Excel file (add, case of multiple spreadsheets)